In [2]:
import pandas as pd
import numpy as np
from pandas_ods_reader import read_ods
#import glob
import os

In [3]:
def pdbqtModelExtract(input_file,Model_No,output_file):
    f = open(input_file,"r")
    tx = f.read()
    txt = tx.replace("MODEL", "&+MODEL")
    txt_l = txt.split("&+")
    f.close()
    nex_f = open(output_file, "w")#.read()
    nex_f.write(txt_l[Model_No])
    nex_f.close()
    

In [6]:
#original info data from ods file
df2 = read_ods("Dataset_Info.ods", 2)
df3 = read_ods("Dataset_Info.ods", 3)
#choose ligand pose used in multi-pose model
df_decoy = df3[df3["Use_in_models_58"] == 1]
df_rdkit = df2[df2["Use_in_models_58"] == 1]
df_rdkit = df_rdkit.rename(columns={"PDBCode":"PDB code"})
df_PDB_ori = pd.concat([df_rdkit[["Pose name","PDB code"]],df_decoy[["Pose name","PDB code"]]], axis=0, ignore_index=True)
#save the info data(pose_name and pdb_code) for further usage
df_PDB_ori.to_csv("Selected_PDBpose_info.csv", index=False)

In [10]:
#read in info data and group by PDB_ID
df_PDB_ori = pd.read_csv("Selected_PDBpose_info.csv")
df_PDB["Pose name"] = df_PDB["Pose name"].apply(lambda x: [x])
df_PDB = df_PDB.groupby("PDB code")["Pose name"].apply(lambda x:np.concatenate(list(x))).reset_index()
df_PDB = df_PDB.set_index("PDB code")
df_PDB

,Pose name
PDB code,
PDB_10gs,"[10gs_rdkit_ligand_01, 10gs_rdkit_ligand_04, 1..."
PDB_184l,"[184l_rdkit_ligand_01, 184l_rdkit_ligand_02, 1..."
PDB_185l,"[185l_rdkit_ligand_01, 185l_rdkit_ligand_02, 1..."
PDB_186l,"[186l_rdkit_ligand_01, 186l_rdkit_ligand_02, 1..."
PDB_187l,"[187l_rdkit_ligand_01, 187l_rdkit_ligand_02, 1..."
...,...
PDB_7std,"[7std_rdkit_ligand_01, 7std_rdkit_ligand_03, 7..."
PDB_7upj,"[7upj_rdkit_ligand_01, 7upj_rdkit_ligand_02, 7..."
PDB_8cpa,"[8cpa_rdkit_ligand_04, 8cpa_rdkit_ligand_07, 8..."


In [11]:
#test pose index info
df_PDB[:2]

,Pose name
PDB code,
PDB_10gs,"[10gs_rdkit_ligand_01, 10gs_rdkit_ligand_04, 1..."
PDB_184l,"[184l_rdkit_ligand_01, 184l_rdkit_ligand_02, 1..."


In [12]:
# pose extraction on the test set
for pdb in df_PDB[:2].index:
    print("Done ",pdb)
    File_path = "./Docked_PDBQTs_test/"+pdb.split("_")[1]+"/ligands/"
    Output_path = "./Docked_PDBQTs_out/"+pdb.split("_")[1]+"/"
    if not os.path.exists(Output_path):
        os.makedirs(Output_path)
    pose_names = df_PDB.loc[pdb][0]
    for pose_name in pose_names:
        model_No = int(pose_name[-2:])
        if "rdkit" in pose_name:
            file_name = File_path + pose_name[:-2] + "out.pdbqt"
        if "decoy" in pose_name:
            file_name = File_path + pose_name[:11]+str(int(pose_name[11:13]))+"_ligand.pdbqt"
        #extrate the listed No. of model from the file_name
        file_extract = Output_path + pose_name + ".pdbqt"
        pdbqtModelExtract(file_name,model_No,file_extract)
    #check file number for each pdb ID:
    
    file_ext_num = len(os.listdir(Output_path))
    assert file_ext_num == len(pose_names)            


Done  PDB_10gs
Done  PDB_184l
